# 1 - Imports

In [108]:
import warnings
import openpyxl
import pyodbc
import numpy                    as np
import pandas                   as pd
import seaborn                  as sns
import matplotlib.pyplot        as plt
import plotly.express           as px

warnings.filterwarnings('ignore')


# 2 - Funções de apoio

In [109]:
# ===================================================================
# Configura o notebook
def jupyter_settings():
    import warnings
    warnings.filterwarnings('ignore')
    %matplotlib inline

    # Tamanho e estilo dos gráficos
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [22, 9]
    plt.rcParams['font.size'] = 21

    # Configuração de exibição das linhas e colunas do pandas
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)

    # configuração do pandas para quantidade de casas decimais
    pd.set_option('display.float_format', lambda x: '%.2f' % x)

    sns.set()

# 3 - Aquisição dos dados

## 3.1 - Conexão com o banco

In [110]:
# Credenciais fornecidas
connection = pyodbc.connect( 
    'Driver={SQL Server};'
    'Server=10.67.99.10\MSSQLSERVER2019;'
    'Database=DATA_WAREHOUSE;'
    'UID=userCustomerServices;'
    'PWD=userCustomerServices@ADT_2023*;'
    'Trusted_Connection=no;')

## 3.2 - Queries

In [ ]:
data_inicial = "'" + input('Informe a data inicial') + "'"
data_final = "'" + input('Informe a data final') + "'"

In [111]:
Q1 = '''
/*  =======================================================================
        QUERY - Pega clientes e data fidelidade 
    ======================================================================= */
WITH CLIENTES as (
    SELECT 
        IdCliente,
        NmCliente,     
        DtRefFidelidade,
        TxRamo,
        IdSituacao
              
    FROM [DATA_WAREHOUSE].[dbo].[Clientes]
    WHERE CONVERT(date, DtRefFidelidade) BETWEEN '''

Q2 = ''' AND '''
Q3 = '''

),

/*  =======================================================================
        QUERY - Pega os valores do contrato de aquisição
    ======================================================================= */
VALOR as (
    SELECT 
        IdCliente,
        VlContrato,
        Convert(date, DtAtivacao) AS DtAtivacao
    FROM DATA_WAREHOUSE.dbo.ClientesServicos 
),

/*  =======================================================================
        QUERY - Pega os informação das vendas (canal, tipo, coordenador)
    ======================================================================= */
TIPO AS (
    SELECT 
        IdCliente,
        TpCanalVendas,
        TipoLead,
        CanalVendas,
        CoordenadorVendas
    FROM [DATA_WAREHOUSE].[dbo].[VendasGeral]

),

/*  =======================================================================
        QUERY - Pega a situação do cliente
    ======================================================================= */
SITUACAO AS (
    SELECT 
        IdSituacao,
        Descricao
    FROM [DATA_WAREHOUSE].[dbo].[Dim_ClienteSituacao]
)

/*  =======================================================================
        QUERY - Junta todas as tabelas
    ======================================================================= */
SELECT 
    CLIENTES.IdCliente,
    MAX(SITUACAO.Descricao)         AS Descricao,
    MAX(CLIENTES.DtRefFidelidade)   AS DtRefFidelidade,
    SUM(VALOR.VlContrato)           AS VlContrato,
    MAX(CLIENTES.TxRamo)            AS TxRamo,
    MAX(TIPO.TpCanalVendas)         AS TpCanalVendas,
    MAX(TIPO.TipoLead)              AS TipoLead,
    MAX(TIPO.CanalVendas)           AS CanalVendas,
    MAX(TIPO.CoordenadorVendas)     AS CoordenadorVendas
FROM 
    CLIENTES
INNER JOIN VALOR        ON CLIENTES.IdCliente = VALOR.IdCliente AND CLIENTES.DtRefFidelidade = VALOR.DtAtivacao
INNER JOIN TIPO         ON CLIENTES.IdCliente = TIPO.IdCliente 
INNER JOIN SITUACAO     ON CLIENTES.IdSituacao = SITUACAO.IdSituacao 

GROUP BY 
    CLIENTES.IdCliente

ORDER BY
    CLIENTES.IdCliente
'''

QUERY = Q1 + data_inicial + Q2 + data_final + Q3


In [112]:
print(data_inicial)
print(data_final)
print(QUERY)

'2023-09-01'
'2023-09-30'

/*  =======================================================================
        QUERY - Pega clientes e data fidelidade 
    ======================================================================= */
WITH CLIENTES as (
    SELECT 
        IdCliente,
        NmCliente,     
        DtRefFidelidade,
        TxRamo,
        IdSituacao
              
    FROM [DATA_WAREHOUSE].[dbo].[Clientes]
    WHERE CONVERT(date, DtRefFidelidade) BETWEEN '2023-09-01' AND '2023-09-30'

),

/*  =======================================================================
        QUERY - Pega os valores do contrato de aquisição
    ======================================================================= */
VALOR as (
    SELECT 
        IdCliente,
        VlContrato,
        Convert(date, DtAtivacao) AS DtAtivacao
    FROM DATA_WAREHOUSE.dbo.ClientesServicos 
),

/*  =======================================================================
        QUERY - Pega os informação das vendas

In [113]:

QUERY_ANTIGA = '''
/*  =======================================================================
        QUERY - Pega clientes e data fidelidade 
    ======================================================================= */
WITH CLIENTES as (
    SELECT 
        IdCliente,
        NmCliente,     
        DtRefFidelidade,
        TxRamo,
        IdSituacao
              
    FROM [DATA_WAREHOUSE].[dbo].[Clientes]
    WHERE CONVERT(date, DtRefFidelidade) BETWEEN '2023-09-01' AND '2023-09-30'

),

/*  =======================================================================
        QUERY - Pega os valores do contrato de aquisição
    ======================================================================= */
VALOR as (
    SELECT 
        IdCliente,
        VlContrato,
        Convert(date, DtAtivacao) AS DtAtivacao
    FROM DATA_WAREHOUSE.dbo.ClientesServicos 
),

/*  =======================================================================
        QUERY - Pega os informação das vendas (canal, tipo, coordenador)
    ======================================================================= */
TIPO AS (
    SELECT 
        IdCliente,
        TpCanalVendas,
        TipoLead,
        CanalVendas,
        CoordenadorVendas
    FROM [DATA_WAREHOUSE].[dbo].[VendasGeral]

),

/*  =======================================================================
        QUERY - Pega a situação do cliente
    ======================================================================= */
SITUACAO AS (
    SELECT 
        IdSituacao,
        Descricao
    FROM [DATA_WAREHOUSE].[dbo].[Dim_ClienteSituacao]
)

/*  =======================================================================
        QUERY - Junta todas as tabelas
    ======================================================================= */
SELECT 
    CLIENTES.IdCliente,
    MAX(SITUACAO.Descricao)         AS Descricao,
    MAX(CLIENTES.DtRefFidelidade)   AS DtRefFidelidade,
    SUM(VALOR.VlContrato)           AS VlContrato,
    MAX(CLIENTES.TxRamo)            AS TxRamo,
    MAX(TIPO.TpCanalVendas)         AS TpCanalVendas,
    MAX(TIPO.TipoLead)              AS TipoLead,
    MAX(TIPO.CanalVendas)           AS CanalVendas,
    MAX(TIPO.CoordenadorVendas)     AS CoordenadorVendas
FROM 
    CLIENTES
INNER JOIN VALOR        ON CLIENTES.IdCliente = VALOR.IdCliente AND CLIENTES.DtRefFidelidade = VALOR.DtAtivacao
INNER JOIN TIPO         ON CLIENTES.IdCliente = TIPO.IdCliente 
INNER JOIN SITUACAO     ON CLIENTES.IdSituacao = SITUACAO.IdSituacao 

GROUP BY 
    CLIENTES.IdCliente

ORDER BY
    CLIENTES.IdCliente
'''

## 3.3 - Pegando os dados do banco

In [114]:
df = pd.read_sql(QUERY, connection)
connection.close()

df.TxRamo = df.TxRamo.apply(lambda x: 'Residencial' if x=='CONDOMINIAL' else str.capitalize(x))

In [115]:
df.head()

,IdCliente,Descricao,DtRefFidelidade,VlContrato,TxRamo,TpCanalVendas,TipoLead,CanalVendas,CoordenadorVendas
0,340450,ATIVO -NAO MONITORADO C/V,2023-09-04,65.00,Empresarial,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO
1,340611,ATIVO-INSTALADO + FICHA,2023-09-05,318.40,Empresarial,Venda Interna,Trabalho Próprio,LEANDRO DOS SANTOS SOUZA,LUIZ CARLOS OTOBONI SPERANDIO ...
2,340631,ATIVO -NAO MONITORADO C/V,2023-09-02,72.61,Empresarial,Venda Interna,Lead,CAMILA DUARTE GONCALVES LIMA,None
3,340654,ATIVO -NAO MONITORADO C/V,2023-09-04,133.36,Empresarial,Venda Interna,Trabalho Próprio,BIANCA GOMES DA SILVEIRA,LUIZ CARLOS OTOBONI SPERANDIO ...
4,340738,ATIVO -NAO MONITORADO C/V,2023-09-04,90.86,Empresarial,Venda Interna,Trabalho Próprio,WASHINGTON ALVES TEIXEIRA JUNIOR,LUIZ CARLOS OTOBONI SPERANDIO ...


## 3.4 - Pegando dados de metas (planilha)

In [116]:
metas = pd.read_excel('../dados/METAS.xlsx')
metas.head()

,Canal de Venda,Meta
0,ALEXANDRE JOSE BARBOSA,14
1,ALINE DA SILVA SANTOS,6
2,ANDERSON MIRANDA BASTOS,8
3,ANDREIA DE SOUZA SILVA,3
4,BIANCA GOMES DA SILVEIRA,5


## 3.5 - Criando a base de atingimento por vendedor

In [117]:
# Agrupa para pegar a contagem de valores
df_faixa_vendedor = df[["CanalVendas", "IdCliente"]].groupby("CanalVendas").count().reset_index()

# Renomeia as colunas
df_faixa_vendedor.columns = ["Vendedor", "Vendas"]

# Junta os dois dataframes
df_faixa_vendedor = pd.merge(df_faixa_vendedor, metas, left_on="Vendedor", right_on="Canal de Venda", how="left").drop(columns=["Canal de Venda"])

# Cria coluna de Atingimento
df_faixa_vendedor["Atingimento"] = df_faixa_vendedor.Vendas / df_faixa_vendedor.Meta
df_faixa_vendedor["Atingimento"].fillna(0, inplace=True)

# Define a coluna de faixas de valores
df_faixa_vendedor["Faixa"] = df_faixa_vendedor.Atingimento.apply(
    lambda x: "Meta não informada"
    if x==0
    else "< 59,99%"
    if x < 0.6
    else "< 69,99%"
    if x < 0.7
    else "< 79,99%"
    if x < 0.8
    else "< 89,99%"
    if x < 0.9
    else "< 99,99%"
    if x < 1
    else "< 109,99%"
    if x < 1.1
    else "< 119,99%"
    if x < 1.2
    else "< 129,99%"
    if x < 1.3
    else "< 139,99%"
    if x < 1.4
    else "< 79,99%"
)

# Define a coluna de faixas de valores (somente MPP)
df_faixa_vendedor["MPP"] = df_faixa_vendedor.Vendas.apply(
    lambda x: "< 11"
    if x < 11
    else "< 21"
    if x < 21
    else "< 31"
    if x < 31
    else "< 61"
    if x < 61
    else "< 81"
    if x < 81
    else "< 131"
    if x < 131
    else "< 150"
    if x < 150
    else ">= 150"
)

# Junta a tabela de dados com a tabela de faixas
df_dados_faixa = df.merge(df_faixa_vendedor, left_on='CanalVendas', right_on='Vendedor', how='inner').drop(columns=['Vendedor'])


# 4 - Cálculo do fator

## 4.1 - Cálculo do vendedor

In [118]:
faixas_percentuais_vendedor = {
    "Venda InternaTrabalho PróprioResidencial< 59,99%": 0.5,
    "Venda InternaTrabalho PróprioResidencial< 69,99%": 0.7,
    "Venda InternaTrabalho PróprioResidencial< 79,99%": 0.8,
    "Venda InternaTrabalho PróprioResidencial< 89,99%": 0.9,
    "Venda InternaTrabalho PróprioResidencial< 99,99%": 1,
    "Venda InternaTrabalho PróprioResidencial< 109,99%": 2,
    "Venda InternaTrabalho PróprioResidencial< 119,99%": 2.3,
    "Venda InternaTrabalho PróprioResidencial< 129,99%": 2.6,
    "Venda InternaTrabalho PróprioResidencial< 139,99%": 2.9,
    "Venda InternaTrabalho PróprioResidencial> 140": 3.2,
    "Venda InternaTrabalho PróprioEmpresarial< 59,99%": 0.3,
    "Venda InternaTrabalho PróprioEmpresarial< 69,99%": 0.5,
    "Venda InternaTrabalho PróprioEmpresarial< 79,99%": 0.6,
    "Venda InternaTrabalho PróprioEmpresarial< 89,99%": 0.7,
    "Venda InternaTrabalho PróprioEmpresarial< 99,99%": 0.8,
    "Venda InternaTrabalho PróprioEmpresarial< 109,99%": 1,
    "Venda InternaTrabalho PróprioEmpresarial< 119,99%": 1.4,
    "Venda InternaTrabalho PróprioEmpresarial< 129,99%": 1.8,
    "Venda InternaTrabalho PróprioEmpresarial< 139,99%": 2.2,
    "Venda InternaTrabalho PróprioEmpresarial> 140": 2.5,
    "Venda InternaLead< 59,99%": 0.1,
    "Venda InternaLead< 69,99%": 0.2,
    "Venda InternaLead< 79,99%": 0.3,
    "Venda InternaLead< 89,99%": 0.4,
    "Venda InternaLead< 99,99%": 0.5,
    "Venda InternaLead< 109,99%": 0.6,
    "Venda InternaLead< 119,99%": 0.8,
    "Venda InternaLead< 129,99%": 1,
    "Venda InternaLead< 139,99%": 1.2,
    "Venda InternaLead> 140": 1.5,
    "TelevendasTrabalho PróprioResidencial< 59,99%": 0.3,
    "TelevendasTrabalho PróprioResidencial< 69,99%": 0.5,
    "TelevendasTrabalho PróprioResidencial< 79,99%": 0.6,
    "TelevendasTrabalho PróprioResidencial< 89,99%": 0.7,
    "TelevendasTrabalho PróprioResidencial< 99,99%": 0.8,
    "TelevendasTrabalho PróprioResidencial< 109,99%": 1,
    "TelevendasTrabalho PróprioResidencial< 119,99%": 1.4,
    "TelevendasTrabalho PróprioResidencial< 129,99%": 1.8,
    "TelevendasTrabalho PróprioResidencial< 139,99%": 2.2,
    "TelevendasTrabalho PróprioResidencial> 140": 2.5,
    "TelevendasTrabalho PróprioEmpresarial< 59,99%": 0.3,
    "TelevendasTrabalho PróprioEmpresarial< 69,99%": 0.5,
    "TelevendasTrabalho PróprioEmpresarial< 79,99%": 0.6,
    "TelevendasTrabalho PróprioEmpresarial< 89,99%": 0.7,
    "TelevendasTrabalho PróprioEmpresarial< 99,99%": 0.8,
    "TelevendasTrabalho PróprioEmpresarial< 109,99%": 1,
    "TelevendasTrabalho PróprioEmpresarial< 119,99%": 1.4,
    "TelevendasTrabalho PróprioEmpresarial< 129,99%": 1.8,
    "TelevendasTrabalho PróprioEmpresarial< 139,99%": 2.2,
    "TelevendasTrabalho PróprioEmpresarial> 140": 2.5,
    "TelevendasLead< 59,99%": 0.3,
    "TelevendasLead< 69,99%": 0.5,
    "TelevendasLead< 79,99%": 0.6,
    "TelevendasLead< 89,99%": 0.7,
    "TelevendasLead< 99,99%": 0.8,
    "TelevendasLead< 109,99%": 1,
    "TelevendasLead< 119,99%": 1.4,
    "TelevendasLead< 129,99%": 1.8,
    "TelevendasLead< 139,99%": 2.2,
    "TelevendasLead> 140": 2.5,
    "MPP< 11": 3,
    "MPP< 21": 3.5,
    "MPP< 31": 4,
    "MPP< 61": 4,
    "MPP< 81": 4,
    "MPP< 131": 4.5,
    "MPP< 150": 4.5,
    "MPP>= 150": 4.5,
}

# Adequando a coluna TxRamo para omitir os dados caso seja MPP ou Lead
df_dados_faixa.TxRamo = df_dados_faixa.apply(lambda x: '' if (x.TpCanalVendas == 'MPP') | (x.TipoLead == 'Lead') else x.TxRamo, axis=1)

# Adequando a coluna TpCanalVendas para omitir o valor 'Corporate'
df_dados_faixa.TpCanalVendas = df_dados_faixa.TpCanalVendas.apply(lambda x: '' if x == 'Corporate' else x)

In [119]:
# Calculando o percentual da comissão
df_dados_faixa["Comissão %"] = df_dados_faixa.apply(
    lambda x: faixas_percentuais_vendedor[x.TpCanalVendas + x.MPP]
    if x.TpCanalVendas == "MPP"
    else 0
    if (x.CanalVendas == "CORPORATIVO - ADT") | (x.TpCanalVendas == 'Supervisor') | (x.TpCanalVendas == 'E-Commerce') | (x.Faixa == "Meta não informada")
    else faixas_percentuais_vendedor[x.TpCanalVendas + x.TipoLead + x.TxRamo + x.Faixa],
    axis=1,
)

# Calculando o valor da comissão
df_dados_faixa['Comissão Valor'] = df_dados_faixa['VlContrato'] * df_dados_faixa['Comissão %']



In [120]:
# Mesclando a planilha de campanha
df_campanha = pd.read_excel('../dados/CAMPANHA.xlsx')
df_final_vendedor = df_dados_faixa.merge(df_campanha, left_on='CanalVendas', right_on='Parceiro', how='left').drop(columns=['Parceiro'])
df_final_vendedor['Termo Aditivo'].fillna('Não', inplace=True)

# Calculando campanha
df_final_vendedor['Valor Campanha'] = df_final_vendedor.apply(lambda x: 0 if x['Termo Aditivo'] == 'Não' else 3 * x['VlContrato'], axis=1)

# Calculando valor total
df_final_vendedor['Valor Comissão Vendedor'] = df_final_vendedor['Valor Campanha'] + df_final_vendedor['Comissão Valor']


In [121]:
df_final_vendedor.head()

,IdCliente,Descricao,DtRefFidelidade,VlContrato,TxRamo,TpCanalVendas,TipoLead,CanalVendas,CoordenadorVendas,Vendas,Meta,Atingimento,Faixa,MPP,Comissão %,Comissão Valor,Termo Aditivo,Valor Campanha,Valor Comissão Vendedor
0,340450,ATIVO -NAO MONITORADO C/V,2023-09-04,65.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8,117.0,Não,0.0,117.0
1,340878,ATIVO -NAO MONITORADO C/V,2023-09-01,120.0,Residencial,Televendas,Trabalho Próprio,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8,216.0,Não,0.0,216.0
2,340913,ATIVO-INSTALADO + FICHA,2023-09-01,345.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8,621.0,Não,0.0,621.0
3,340935,ATIVO-INSTALADO + FICHA,2023-09-06,150.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8,270.0,Não,0.0,270.0
4,340948,ATIVO -NAO MONITORADO C/V,2023-09-09,125.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8,225.0,Não,0.0,225.0


## 4.2 - Cálculo do Supervisor

In [122]:
# Encontrar os vendedores e as respectivas metas, agrupados por supervisor
df_metas = df_final_vendedor[['CanalVendas', 'CoordenadorVendas', 'Meta']].groupby(['CanalVendas', 'CoordenadorVendas']).max().reset_index()
# Somar as metas individuais dos vendedores, por supervisor
df_metas = df_metas[['CoordenadorVendas', 'Meta']].groupby('CoordenadorVendas').sum().reset_index()
# Renomear colunas
df_metas.columns = ['CoordenadorVendas',	'Meta Supervisor']
df_metas

,CoordenadorVendas,Meta Supervisor
0,DOUGLAS SANTOS MEDEIROS,29.0
1,ELIEL DOS SANTOS LIMA,0.0
2,LUIZ CARLOS OTOBONI SPERANDIO ...,133.0
3,MAURICIO CAPARROZ,0.0
4,MIGUEL VETTORATO NETO,69.0


In [123]:
# Encontrar os vendedores e as respectivas vendas, agrupados por supervisor
df_vendas = df_final_vendedor[['CanalVendas', 'CoordenadorVendas', 'Vendas']].groupby(['CanalVendas', 'CoordenadorVendas']).max().reset_index()
display(df_vendas)
# Somar as vendas individuais dos vendedores, por supervisor
df_vendas = df_vendas[['CoordenadorVendas', 'Vendas']].groupby('CoordenadorVendas').sum().reset_index()
# Renomear colunas
df_vendas.columns = ['CoordenadorVendas',	'Vendas Supervisor']
df_vendas

,CanalVendas,CoordenadorVendas,Vendas
0,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5
1,ANDREIA DE SOUZA SILVA\t\t,LUIZ CARLOS OTOBONI SPERANDIO ...,3
2,BIANCA GOMES DA SILVEIRA,LUIZ CARLOS OTOBONI SPERANDIO ...,1
3,BIANCA LAUREANO DE SOUZA,LUIZ CARLOS OTOBONI SPERANDIO ...,1
4,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19
5,CAMILA GONCALVES DA SILVA,MIGUEL VETTORATO NETO,6
6,CARLA GONCALVES PAIVA,MIGUEL VETTORATO NETO,3
7,CAROLINE BARBOZA LOPES,ELIEL DOS SANTOS LIMA,5
8,DL LITORAL LTDA,ELIEL DOS SANTOS LIMA,1
9,E-COMMERCE,MIGUEL VETTORATO NETO,1


,CoordenadorVendas,Vendas Supervisor
0,DOUGLAS SANTOS MEDEIROS,30
1,ELIEL DOS SANTOS LIMA,19
2,LUIZ CARLOS OTOBONI SPERANDIO ...,147
3,MAURICIO CAPARROZ,7
4,MIGUEL VETTORATO NETO,71


In [124]:
# Juntando as tabelas
df_metas = df_vendas.merge(df_metas, on='CoordenadorVendas', how='inner')
display(df_metas)

# Juntando a meta do supervisor ao aos demais dados
df_final_supervisor = df_final_vendedor.merge(df_metas, on='CoordenadorVendas', how='inner')
df_final_supervisor.sort_values('CanalVendas').head()

,CoordenadorVendas,Vendas Supervisor,Meta Supervisor
0,DOUGLAS SANTOS MEDEIROS,30,29.0
1,ELIEL DOS SANTOS LIMA,19,0.0
2,LUIZ CARLOS OTOBONI SPERANDIO ...,147,133.0
3,MAURICIO CAPARROZ,7,0.0
4,MIGUEL VETTORATO NETO,71,69.0


,IdCliente,Descricao,DtRefFidelidade,VlContrato,TxRamo,TpCanalVendas,TipoLead,CanalVendas,CoordenadorVendas,Vendas,...,Atingimento,Faixa,MPP,Comissão %,Comissão Valor,Termo Aditivo,Valor Campanha,Valor Comissão Vendedor,Vendas Supervisor,Meta Supervisor
69,341169,ATIVO-INSTALADO + FICHA,2023-09-27,227.0,,Televendas,Lead,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5,...,0.833333,"< 89,99%",< 11,0.7,158.90,Não,0.0,158.90,71,69.0
68,341106,ATIVO-INSTALADO + FICHA,2023-09-20,602.5,,Televendas,Lead,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5,...,0.833333,"< 89,99%",< 11,0.7,421.75,Não,0.0,421.75,71,69.0
67,341068,ATIVO -NAO MONITORADO C/V,2023-09-23,205.0,,Televendas,Lead,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5,...,0.833333,"< 89,99%",< 11,0.7,143.50,Não,0.0,143.50,71,69.0
66,341037,ATIVO -NAO MONITORADO C/V,2023-09-12,183.0,,Televendas,Lead,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5,...,0.833333,"< 89,99%",< 11,0.7,128.10,Não,0.0,128.10,71,69.0
65,341033,ATIVO-INSTALADO + FICHA,2023-09-12,205.0,,Televendas,Lead,ALINE DA SILVA SANTOS,MIGUEL VETTORATO NETO,5,...,0.833333,"< 89,99%",< 11,0.7,143.50,Não,0.0,143.50,71,69.0


In [125]:
# valor
df_final_supervisor['Atingimento Supervisor'] = np.where((df_final_supervisor['Meta Supervisor'] == 0), 0, df_final_supervisor['Vendas Supervisor'] /	df_final_supervisor['Meta Supervisor'])
df_final_supervisor.head()

,IdCliente,Descricao,DtRefFidelidade,VlContrato,TxRamo,TpCanalVendas,TipoLead,CanalVendas,CoordenadorVendas,Vendas,...,Faixa,MPP,Comissão %,Comissão Valor,Termo Aditivo,Valor Campanha,Valor Comissão Vendedor,Vendas Supervisor,Meta Supervisor,Atingimento Supervisor
0,340450,ATIVO -NAO MONITORADO C/V,2023-09-04,65.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,...,"< 129,99%",< 21,1.8,117.0,Não,0.0,117.0,71,69.0,1.028986
1,340878,ATIVO -NAO MONITORADO C/V,2023-09-01,120.0,Residencial,Televendas,Trabalho Próprio,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,...,"< 129,99%",< 21,1.8,216.0,Não,0.0,216.0,71,69.0,1.028986
2,340913,ATIVO-INSTALADO + FICHA,2023-09-01,345.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,...,"< 129,99%",< 21,1.8,621.0,Não,0.0,621.0,71,69.0,1.028986
3,340935,ATIVO-INSTALADO + FICHA,2023-09-06,150.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,...,"< 129,99%",< 21,1.8,270.0,Não,0.0,270.0,71,69.0,1.028986
4,340948,ATIVO -NAO MONITORADO C/V,2023-09-09,125.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,...,"< 129,99%",< 21,1.8,225.0,Não,0.0,225.0,71,69.0,1.028986


In [126]:
# Define a coluna de faixas de valores para os supervisores
df_final_supervisor["Faixa Supervisor"] = df_final_supervisor['Atingimento Supervisor'].apply(
    lambda x: "< 49,99%"
    if x < .5
    else "< 59,99%"
    if x < 0.6
    else "< 69,99%"
    if x < 0.7
    else "< 79,99%"
    if x < 0.8
    else "< 89,99%"
    if x < 0.9
    else "< 99,99%"
    if x < 1
    else "< 109,99%"
    if x < 1.1
    else "< 119,99%"
    if x < 1.2
    else "< 129,99%"
    if x < 1.3
    else "> 130%"
)

# Define a coluna de faixas de valores para os Supervisores (somente MPP)
df_final_supervisor["MPP Supervisor"] = df_final_supervisor['Atingimento Supervisor'].apply(
    lambda x: "< 79,99%"
    if x < 0.8
    else "< 89,99%"
    if x < 0.9
    else "< 99,99%"
    if x < 1
    else "< 109,99%"
    if x < 1.1
    else "< 119,99%"
    if x < 1.2
    else "< 129,99%"
    if x < 1.3
    else "< 139,99%"
    if x < 1.4
    else '> 140%'
)

df_final_supervisor['faixa concatenada'] = df_final_supervisor.apply(lambda x: x['MPP Supervisor'] if x['TpCanalVendas'] == 'MPP' else x['Faixa Supervisor'], axis=1)

In [127]:
faixas_percentuais_supervisor = {
    "Venda Interna< 49,99%": 0,
    "Venda Interna< 59,99%": 0.04,
    "Venda Interna< 69,99%": 0.06,
    "Venda Interna< 79,99%": 0.07,
    "Venda Interna< 89,99%": 0.08,
    "Venda Interna< 99,99%": 0.09,
    "Venda Interna< 109,99%": 0.12,
    "Venda Interna< 119,99%": 0.14,
    "Venda Interna< 129,99%": 0.15,
    "Venda Interna> 130%": 0.17,
    "Televendas< 49,99%": 0,
    "Televendas< 59,99%": 0.04,
    "Televendas< 69,99%": 0.06,
    "Televendas< 79,99%": 0.07,
    "Televendas< 89,99%": 0.08,
    "Televendas< 99,99%": 0.09,
    "Televendas< 109,99%": 0.12,
    "Televendas< 119,99%": 0.14,
    "Televendas< 129,99%": 0.15,
    "Televendas> 130%": 0.17,
    "MPP< 79,99%": 0,
    "MPP< 89,99%": 0.06,
    "MPP< 99,99%": 0.1,
    "MPP< 109,99%": 0.16,
    "MPP< 119,99%": 0.19,
    "MPP< 129,99%": 0.22,
    "MPP< 1399,99%": 0.25,
    "MPP> 140%": 0.3,
}

In [128]:
#  Calculando o percentual da comissão
df_final_supervisor['Comissão % Supervisor'] = df_final_supervisor.apply(
    lambda x: faixas_percentuais_supervisor[ x['TpCanalVendas'] + x['faixa concatenada'] ]
    if x.TpCanalVendas not in ["E-Commerce", 'Corporate', 'Supervisor']
    else 0,    
    axis=1
)

# Calculando o valor da comissão
df_final_supervisor['Valor Comissão Supervisor'] = df_final_supervisor['Comissão % Supervisor'] * df_final_supervisor['VlContrato']

In [129]:
df_final = df_final_supervisor.copy()

Contratos
- 340928
- 341089

# 5 - Tabela agrupada

In [130]:
# Gera o relatório por Vendedor
df_vendedor_agrupado = df_final_supervisor[['CanalVendas', 'Valor Comissão Vendedor']].groupby('CanalVendas').sum().reset_index()
df_vendedor_agrupado.head()

,CanalVendas,Valor Comissão Vendedor
0,ALINE DA SILVA SANTOS,995.750
1,ANDREIA DE SOUZA SILVA\t\t,0.000
2,BIANCA GOMES DA SILVEIRA,40.008
3,BIANCA LAUREANO DE SOUZA,38.760
4,BRUNO PINHEIRO DE LIMA,7548.840


In [131]:
# Gera o relatório por Supervisor
df_supervisor_agrupado = df_final_supervisor[['CoordenadorVendas', 'Valor Comissão Supervisor']].groupby('CoordenadorVendas').sum().reset_index()
df_supervisor_agrupado.head()

,CoordenadorVendas,Valor Comissão Supervisor
0,DOUGLAS SANTOS MEDEIROS,435.8628
1,ELIEL DOS SANTOS LIMA,0.0000
2,LUIZ CARLOS OTOBONI SPERANDIO ...,3118.6988
3,MAURICIO CAPARROZ,0.0000
4,MIGUEL VETTORATO NETO,1988.6508


In [132]:
# Gera o relatório geral
display(df_final.iloc[:,:15].head())
display(df_final.iloc[:,15:].head())


,IdCliente,Descricao,DtRefFidelidade,VlContrato,TxRamo,TpCanalVendas,TipoLead,CanalVendas,CoordenadorVendas,Vendas,Meta,Atingimento,Faixa,MPP,Comissão %
0,340450,ATIVO -NAO MONITORADO C/V,2023-09-04,65.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8
1,340878,ATIVO -NAO MONITORADO C/V,2023-09-01,120.0,Residencial,Televendas,Trabalho Próprio,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8
2,340913,ATIVO-INSTALADO + FICHA,2023-09-01,345.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8
3,340935,ATIVO-INSTALADO + FICHA,2023-09-06,150.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8
4,340948,ATIVO -NAO MONITORADO C/V,2023-09-09,125.0,,Televendas,Lead,BRUNO PINHEIRO DE LIMA,MIGUEL VETTORATO NETO,19,15.0,1.266667,"< 129,99%",< 21,1.8


,Comissão Valor,Termo Aditivo,Valor Campanha,Valor Comissão Vendedor,Vendas Supervisor,Meta Supervisor,Atingimento Supervisor,Faixa Supervisor,MPP Supervisor,faixa concatenada,Comissão % Supervisor,Valor Comissão Supervisor
0,117.0,Não,0.0,117.0,71,69.0,1.028986,"< 109,99%","< 109,99%","< 109,99%",0.12,7.8
1,216.0,Não,0.0,216.0,71,69.0,1.028986,"< 109,99%","< 109,99%","< 109,99%",0.12,14.4
2,621.0,Não,0.0,621.0,71,69.0,1.028986,"< 109,99%","< 109,99%","< 109,99%",0.12,41.4
3,270.0,Não,0.0,270.0,71,69.0,1.028986,"< 109,99%","< 109,99%","< 109,99%",0.12,18.0
4,225.0,Não,0.0,225.0,71,69.0,1.028986,"< 109,99%","< 109,99%","< 109,99%",0.12,15.0


In [133]:
df_supervisor_agrupado.to_excel('../dados/relatorios/Comissão Supervisor.xlsx', index=False)
df_vendedor_agrupado.to_excel('../dados/relatorios/Comissão Vendedor.xlsx', index=False)
df_final.to_excel('../dados/relatorios/Planilha Geral.xlsx', index=False)